# 光流方法

In [4]:
import numpy as np
import cv2 as cv
cap = cv.VideoCapture('klsq20201214115453.mp4')
cap.set(2,0)
fraspd = cap.get(5)
out = cv.VideoWriter('out.avi',cv.VideoWriter_fourcc(*'MJPG'), fraspd, (int(cap.get(3)),int(cap.get(4)))) # 创建视频写入的对象
ret, frame1 = cap.read()
prvs = cv.cvtColor(frame1,cv.COLOR_BGR2GRAY) # 过去一帧图像
hsv = np.zeros_like(frame1)
hsv[...,1] = 255
while(1):
    ret, frame2 = cap.read()
    next = cv.cvtColor(frame2,cv.COLOR_BGR2GRAY)
    flow = cv.calcOpticalFlowFarneback(prvs,next, None, 0.5, 3, 15, 3, 5, 1.2, 0)
    mag, ang = cv.cartToPolar(flow[...,0], flow[...,1])
    hsv[...,0] = ang*180/np.pi/2
    hsv[...,2] = cv.normalize(mag,None,0,255,cv.NORM_MINMAX)
    bgr = cv.cvtColor(hsv,cv.COLOR_HSV2BGR)
    cv.imshow('frame2',bgr)
    out.write(bgr)
    k = cv.waitKey(int(fraspd)) & 0xff
    if k == 27:
        break
    prvs = next
cap.release()
out.release()
cv.destroyAllWindows()

# 光流+轨迹线

In [6]:
import numpy as np
import cv2 as cv
import argparse


cap = cv.VideoCapture('klsq20201214115453.mp4')
cv.namedWindow('frame', cv.WINDOW_NORMAL) # 调整窗口为可拖动
cv.moveWindow("frame",0,0)

# 用于ShiTomasi拐点检测的参数
feature_params = dict( maxCorners = 100,
                       qualityLevel = 0.3,
                       minDistance = 7,
                       blockSize = 7 )

# lucas kanade光流参数
lk_params = dict( winSize  = (15,15),
                  maxLevel = 2,
                  criteria = (cv.TERM_CRITERIA_EPS | cv.TERM_CRITERIA_COUNT, 10, 0.03))

# 创建一些随机的颜色
color = np.random.randint(0,255,(100,3))

# 拍摄第一帧并在其中找到拐角
ret, old_frame = cap.read()
old_gray = cv.cvtColor(old_frame, cv.COLOR_BGR2GRAY)
p0 = cv.goodFeaturesToTrack(old_gray, mask = None, **feature_params)

# 创建用于作图的掩码图像
mask = np.zeros_like(old_frame)

while(1):
    ret,frame = cap.read()
    frame_gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
    # 计算光流
    p1, st, err = cv.calcOpticalFlowPyrLK(old_gray, frame_gray, p0, None, **lk_params)
    # 选择良好点
    good_new = p1[st == 1]
    good_old = p0[st == 1]
    # 绘制跟踪
    for i,(new,old) in enumerate(zip(good_new, good_old)):
        a,b = new.ravel()
        c,d = old.ravel()
        mask = cv.line(mask, (a,b),(c,d), color[i].tolist(), 2)
        frame = cv.circle(frame,(a,b),5,color[i].tolist(),-1)
    img = cv.add(frame,mask)
    cv.imshow('frame',img)
    k = cv.waitKey(30) & 0xff
    if k == 27:
        break
    # 现在更新之前的帧和点
    old_gray = frame_gray.copy()
    p0 = good_new.reshape(-1,1,2)
cap.release()
cv.destroyAllWindows()